inference

In [1]:
import gc
gc.collect()

0

In [2]:
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation
import numpy as np

In [3]:
def gelu(x):  return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

In [4]:
get_custom_objects().update({'gelu': Activation(gelu)})

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [6]:
# 4)단계와 같은 Sequential 모델을 정의합니다
model = Sequential([
  Dense(units=1024, activation='gelu', input_dim=226),
  Dense(units=900, activation='gelu'),
  Dropout(0.02),
  Dense(units=1024, activation='gelu'),
  Dense(units=512, activation='gelu'),
  Dense(units=512, activation='gelu'),
  Dense(units=4, activation='linear')
])

In [7]:
# 모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [8]:
# 모델(가중치) 로드
import os
# /mnt/output/experiment/output/experiment/model/checkpoints
model_chk_path = os.path.join('/mnt/output/experiment/model', 'checkpoints', 'model.ckpt')
# model_chk_path = os.path.join('output/experiment/model', 'checkpoints', 'model.ckpt')
model.load_weights(model_chk_path)

In [9]:
# 테스트 데이터 로드
import pandas as pd

# data_dir='data/'
data_dir='/mnt/input/root/public/data/thin-film/'

test_data_x = data_dir + 'test-splited-x.csv'
test_data_y = data_dir + 'test-splited-y.csv'

test_x = pd.read_csv(test_data_x)
test_y = pd.read_csv(test_data_y)

In [10]:
test_X = test_x.iloc[:,1:]
test_Y = test_y.iloc[:,1:]

In [11]:
pred_test = model.predict(test_X)

In [12]:
pred_test

array([[ 18.367968,  10.757891,  17.202366,  85.849495],
       [ 19.974007,   9.67644 ,  12.18639 , 237.20172 ],
       [ 13.126524,  14.020803,  27.145195,  43.455776],
       ...,
       [299.5693  , 299.7315  , 190.38782 , 280.5561  ],
       [299.8687  , 300.1924  , 249.9323  ,  99.67269 ],
       [298.85272 , 298.5757  , 281.48486 , 229.37175 ]], dtype=float32)

In [13]:
test_Y

,layer_1,layer_2,layer_3,layer_4
0,10,10,20,90
1,10,10,20,240
2,10,10,40,40
3,10,10,60,280
4,10,10,180,300
5,10,10,200,80
6,10,10,250,50
7,10,10,250,130
8,10,10,250,210
9,10,10,270,150


In [14]:
from tensorflow.keras.losses import MAE

# mae_mean = MAE(test_Y.to_numpy(), pred_test) # pandas 0.24 이후 사용
mae_mean = MAE(test_Y.values, pred_test)

In [15]:
tf.__version__

'2.1.2'

In [16]:
pd.__version__

'0.23.4'

In [17]:
np.mean(mae_mean)

0.53375506